# DeepAgents Research Assistant Tutorial

This notebook demonstrates how to build an AI research agent using DeepAgents, a framework for creating sophisticated AI agents that can conduct deep research, write reports, and critique their own work by using free API's.

## Prerequisites
Before running this notebook, you'll need:
1. A **Tavily API key** (for web search) - Get it free at https://tavily.com/
2. A **Groq API key** (for LLM access) - Get it free at https://console.groq.com/

## What This Notebook Does
- Sets up a research agent that can search the web
- Creates sub-agents for specialized tasks (research and critique)
- Demonstrates how to ask questions and get detailed research reports

## Step 1: Install Required Libraries

First, we need to install the necessary Python packages. This might take a minute.

In [1]:
# Install DeepAgents - the main framework for building research agents
! pip install deepagents -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.2/364.2 kB 9.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.1/369.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.9/223.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.4/357.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 7.1 MB/s eta 0:0

In [2]:
# Install Tavily - used for web search capabilities
! pip install tavily-python -q

In [3]:
# Install LangChain Groq integration - connects to Groq's LLM API
! pip install langchain-groq -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 2.6 MB/s eta 0:00:0000:01


## Step 2: Set Up API Keys

⚠️ **IMPORTANT**: Replace the placeholder text with your actual API keys!

- Get Tavily API key: https://tavily.com/
- Get Groq API key: https://console.groq.com/

In [4]:
import os

# Replace with your actual Tavily API key
os.environ["TAVILY_API_KEY"] = "YOUR TAVILY API KEY HERE"

In [5]:
# Replace with your actual Groq API key
os.environ["GROQ_API_KEY"] = "YOUR GROQ API KEY HERE"

## Step 3: Test Tavily Search (Optional)

Let's verify that Tavily is working correctly by running a simple search.

In [6]:
from tavily import TavilyClient

# Initialize the Tavily client with your API key
tavily_client = TavilyClient(os.environ["TAVILY_API_KEY"])

# Run a test search
search_response = tavily_client.search("Who is Leo Messi?")

# Print the results (you'll see URLs, titles, and content snippets)
print(search_response)

{'query': 'Who is Leo Messi?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.biography.com/athletes/lionel-messi', 'title': 'Lionel Messi: Biography, Soccer Player, Inter Miami CF, Athlete', 'content': 'Lionel Messi, a forward for Inter Miami CF, is one of the world’s greatest soccer players and helped the Argentina national team win its third FIFA World Cup in 2022. Messi, now playing for Inter Miami CF of the MLS, helped his home country win soccer’s biggest event for the first time since 1986, scoring two goals in the final and leading Argentina to a 4-2 win over Kylian Mbappé and France on penalties. Lionel Messi is an Argentinian soccer player who has played for FC Barcelona, Paris Saint-Germain, and currently, the MLS club Inter Miami CF as well as the Argentina national team.', 'score': 0.9928786, 'raw_content': None}, {'url': 'https://www.transfermarkt.us/lionel-messi/profil/spieler/28003', 'title': 'Lionel Messi - Player profile 20

## Step 4: Test Groq LLM (Optional)

Let's verify that the Groq LLM is working by asking it a simple question.

In [7]:
from langchain_groq import ChatGroq

# Initialize the Groq LLM
# Using Llama 4 Scout model with specific parameters
llm = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=1,  # Controls randomness (0=deterministic, 2=very random)
    max_completion_tokens=8192,  # Maximum length of response
    top_p=1,  # Nucleus sampling parameter
)

# Create a test message
messages = [
    {
        "role": "user", 
        "content": "What happened in AI last week? Give me a concise, one paragraph summary of the most important events."
    }
]

# Get the AI's response
ai_msg = llm.invoke(messages)
print(ai_msg.content)

/usr/local/lib/python3.11/dist-packages/pydantic/main.py:250: UserWarning: WARNING! max_completion_tokens is not default parameter.
                    max_completion_tokens was transferred to model_kwargs.
                    Please confirm that max_completion_tokens is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/usr/local/lib/python3.11/dist-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


Last week in AI, Meta unveiled its new Llama 3.1 large language model, boasting 405 billion parameters and a 128k context window. This open model is claimed to outperform competitors like GPT-4o and Claude 3.5 Sonnet on certain tasks. The move marks a significant push into open-source AI by Meta, potentially democratizing access to advanced AI capabilities. Meanwhile, OpenAI's SearchGPT began gaining traction, providing real-time information retrieval and aiming to challenge Google's dominance in search. There were also advancements in AI chip technology, with reports of NVIDIA's upcoming Blackwell Ultra chip and TSMC's plans to boost AI chip packaging capacity, signaling continued investment in AI infrastructure.


## Step 5: Define the Internet Search Tool

This function will be used by our agents to search the web for information.

In [8]:
import os
from typing import Literal

from deepagents import create_deep_agent
from tavily import TavilyClient

# It's best practice to initialize the client once and reuse it.
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])


# Search tool to use to do research
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    search_docs = tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_docs

## Step 6: Configure the Research Agent System

Now we'll set up a sophisticated multi-agent system:
1. **Main Research Agent**: Coordinates the overall research process
2. **Research Sub-Agent**: Conducts deep research on specific topics
3. **Critique Sub-Agent**: Reviews and provides feedback on reports

In [9]:
from langchain_groq import ChatGroq
from deepagents import create_deep_agent

# This agent focuses on conducting thorough research on specific topics
sub_research_prompt = """You are a dedicated researcher. Your job is to conduct research based on the users questions.
Conduct thorough research and then reply to the user with a detailed answer to their question
only your FINAL answer will be passed on to the user. They will have NO knowledge of anything except your final message, so your final report should be your final message!"""

research_sub_agent = {
    "name": "research-agent",
    "description": "Used to research more in depth questions. Only give this researcher one question at a time. Do not pass multiple sub questions to this researcher. Instead, you should break down a large topic into the necessary components, and then call multiple research agents in parallel, one for each sub question.",
    "system_prompt": sub_research_prompt,
    "tools": [internet_search],  # Give this agent access to web search
}


# This agent reviews reports and provides constructive feedback
sub_critique_prompt = """You are a dedicated editor. You are being tasked to critique a report. Be concise and direct.
You can find the report at `final_report.md`.
You can find the question/topic for this report at `question.txt`.
The user may ask for specific areas to critique the report in. Respond to the user with a detailed critique of the report. Things that could be improved.
You can use the search tool to search for information, if that will help you critique the report
Do not write to the `final_report.md` yourself.

Things to check:
- Check that each section is appropriately named
- Check that the report is written as you would find in an essay or a textbook - it should be text heavy, do not let it just be a list of bullet points!
- Check that the report is comprehensive. If any paragraphs or sections are short, or missing important details, point it out.
- Check that the article covers key areas of the industry, ensures overall understanding, and does not omit important parts.
- Check that the article deeply analyzes causes, impacts, and trends, providing valuable insights
- Check that the article closely follows the research topic and directly answers questions
- Check that the article has a clear structure, fluent language, and is easy to understand.
- Be concise and direct.
"""

critique_sub_agent = {
    "name": "critique-agent",
    "description": "Used to critique the final report. Give this agent some information about how you want it to critique the report.",
    "system_prompt": sub_critique_prompt,
}


# This is the orchestrator that manages the research process
research_instructions = """You are an expert researcher. Your job is to conduct thorough research, and then write a polished report.
The first thing you should do is to write the original user question to `question.txt` so you have a record of it.
Use the research-agent to conduct deep research. It will respond to your questions/topics with a detailed answer.
When you think you enough information to write a final report, write it to `final_report.md`
You can call the critique-agent to get a critique of the final report. After that (if needed) you can do more research and edit the `final_report.md`
You can do this however many times you want until are you satisfied with the result.
Only edit the file once at a time (if you call this tool in parallel, there may be conflicts).
Here are instructions for writing the final report
<report_instructions>
CRITICAL: Make sure the answer is written in the same language as the human messages!
Please create a detailed answer to the overall research brief that:
1. Is well-organized with proper headings (# for title, ## for sections, ### for subsections)
2. Includes specific facts and insights from the research
3. References relevant sources using [Title](URL) format
4. Provides a balanced, thorough analysis. Be as comprehensive as possible.
5. Includes a "Sources" section at the end with all referenced links

Format the report in clear markdown with proper structure and include source references where appropriate.
<Citation Rules>
- Assign each unique URL a single citation number in your text
- End with ### Sources that lists each source with corresponding numbers
- Number sources sequentially without gaps (1,2,3,4...)
- Example format:
  [1] Source Title: URL
  [2] Source Title: URL
</Citation Rules>
</report_instructions>
"""


llm = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct", # You can change model according to the requirement from groq model list
    temperature=0.7,
    max_completion_tokens=8192,  # Limit response length for each agent call
    top_p=1,
)


agent = create_deep_agent(
    model=llm,  # The LLM to use
    system_prompt=research_instructions,  # Instructions for the main agent
    subagents=[critique_sub_agent, research_sub_agent],  # Specialized sub-agents
)

print("✅ Research agent system created successfully!")

/usr/local/lib/python3.11/dist-packages/pydantic/main.py:250: UserWarning: WARNING! max_completion_tokens is not default parameter.
                    max_completion_tokens was transferred to model_kwargs.
                    Please confirm that max_completion_tokens is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/usr/local/lib/python3.11/dist-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


✅ Research agent system created successfully!


## Step 7: Ask the Agent a Question

Now we can ask our research agent to investigate a topic. The agent will:
1. Search the web for information
2. Analyze the results
3. Write a comprehensive report
4. Optionally critique and improve the report

**Note**: This may take time depending on the complexity of the question.

In [10]:
# Ask a question - you can change this to any topic you're interested in!
question = "What is LangChain?"

print(f"🔍 Researching: {question}")
print("⏳ Please wait while the agent conducts research...\n")

# Invoke the agent with your question
result = agent.invoke({"messages": [{"role": "user", "content": question}]})

print("\n✅ Research complete!")

🔍 Researching: What is LangChain?
⏳ Please wait while the agent conducts research...


✅ Research complete!


## Step 8: View the Results

Let's examine what the agent discovered and generated.

In [11]:
# Display the full result structure
print("-" * 100)
print("Whole Research Structure")
print("-" * 100)
print(result)
print("\n")

----------------------------------------------------------------------------------------------------
Whole Research Structure
----------------------------------------------------------------------------------------------------
{'messages': [HumanMessage(content='What is LangChain?', additional_kwargs={}, response_metadata={}, id='fc9451cf-b150-4189-8ccf-686b2fa7ef71'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '927ndwqc7', 'function': {'arguments': '{"description":"Research LangChain and provide a detailed description.","subagent_type":"research-agent"}', 'name': 'task'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 5872, 'total_tokens': 5917, 'completion_time': 0.103981433, 'prompt_time': 0.178188149, 'queue_time': 0.070347242, 'total_time': 0.282169582}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_59d473797c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls',

## Step 9: Check Generated Files & Content

If the agent created files (like `question.txt` or `final_report.md`), we can view them here.

In [12]:
# Check if any files were created during the research process
if 'files' in result and result['files']:
    print("📁 Files created by the agent:\n")
    for filepath, file_info in result['files'].items():
        print(f"\n{'-' * 100}")
        print(f"FILE: {filepath}")
        print(f"Created: {file_info['created_at']}")
        print(f"Modified: {file_info['modified_at']}")
        print(f"{'-' * 100}")
        print('\n'.join(file_info['content']))
else:
    print("No files were created during this research session.")

📁 Files created by the agent:


----------------------------------------------------------------------------------------------------
FILE: /question.txt
Created: 2025-11-14T17:54:02.953453+00:00
Modified: 2025-11-14T17:54:02.953453+00:00
----------------------------------------------------------------------------------------------------
# LangChain

LangChain is an open-source framework designed to help developers build applications powered by large language models (LLMs) like ChatGPT, LLaMA, and others. It provides a set of tools and APIs that simplify the process of integrating LLMs into various applications, enabling developers to focus on building innovative and intelligent software solutions.

## Key Features of LangChain:

1. **Modular Architecture**: LangChain has a modular design that allows developers to easily swap out different LLMs, experiment with various models, and integrate multiple models into a single application.
2. **API-First Approach**: The framework provides a si

## Next Steps

Now that you have a working research agent, you can:

1. **Try different questions**: Modify the question in Step 7 and run it again
2. **Customize the agents**: Edit the system prompts to change agent behavior
3. **Adjust parameters**: Try different temperature or max_tokens values
4. **Add more tools**: Create additional functions for the agents to use
5. **Create more sub-agents**: Add specialized agents for specific tasks

## Common Issues & Solutions

- **API Key Errors**: Make sure you've replaced the placeholder text with real API keys
- **Rate Limits**: Free tier APIs have usage limits; wait a few minutes if you hit them
- **Slow Responses**: Complex questions take longer; simpler questions are faster
- **Import Errors**: Restart the kernel and run all cells from the beginning

## Learn More

- DeepAgents GitHub: https://github.com/langchain-ai/deepagents
- LangChain Documentation: https://python.langchain.com/
- Groq Documentation: https://console.groq.com/docs
- Tavily Documentation: https://docs.tavily.com/